In [184]:
import os
import pandas as pd

In [185]:
def get_data_file_path(current_trial_name="v2_eval_20_3", base_folder="game_logs", trial_base_name="alfworld_eval_", csv_file_name="alfworld_results.csv"):
    """returns the csv file path."""
    current_trial_folder = trial_base_name + current_trial_name
    file_path = os.path.join(base_folder, current_trial_folder, csv_file_name)
    return file_path

In [528]:
def convert_string_to_bool(in_string):
    """if string ="" False, else True"""
    if in_string:
        return True
    else:
        return False

def convert_string_to_bool_int(in_string):
    """if string ="" False, else True"""
    if in_string:
        return 1
    else:
        return 0

def convert_bool_string_to_bool_int(in_string):
    """ if string==False -> False, etc."""
    if in_string=="(False,)" or in_string=="False":
        return 0
    elif in_string=="(True,)" or in_string=="True":
        return 1
    elif in_string=="0":
        return 0
    elif in_string=="1":
        return 1
    else:
        print(type(in_string))
        print(in_string)
        print(in_string[0])
        raise Exception("Error in Transforming BoolString to Int")

def modify_text_prompt(keys_removed):
    """changes to short version"""
    keys = keys_removed.count("+")
    if keys==0:
        return keys_removed
    elif keys>1:
        return "short"
    else:
        return "long"

In [529]:
def load_data(current_trial_name, filter_flag=False, collect_results=False, override_filepath=False):
    """Loads the data"""
    if override_filepath:
        file_path = current_trial_name
        full_file_path = file_path
    else:
        file_path = get_data_file_path(current_trial_name=current_trial_name)
        full_file_path = os.path.join("..",file_path)
    results_data = pd.read_csv(
        full_file_path, 
        converters = {
            "early_stop":convert_string_to_bool_int, 
            "error": convert_string_to_bool_int,
            "done": convert_bool_string_to_bool_int,
            "success": convert_bool_string_to_bool_int,
            "keys_removed": modify_text_prompt
        }
    )
    if collect_results:
        clean_results = results_data
    else:
        header_order = ["model", "keys_removed","temperature","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions","error", "env_idx"]
        clean_results = results_data[header_order]
    # Filters for short, long and react-2 prompts
    if filter_flag: 
        filtering_condition = (clean_results["keys_removed"]=="short") | (clean_results["keys_removed"]=="long") | (clean_results["keys_removed"]=="react-2") | (clean_results["keys_removed"]=="react-1")| (clean_results["keys_removed"]=="long-2") | (clean_results["keys_removed"]=="short-2") | (clean_results["keys_removed"]=="short-2-swaped") 
        clean_results = clean_results[ filtering_condition ]
    
    return clean_results

In [542]:
FLAG_ALL = False
FLAG_ALL = True
COLLECT_RESULTS = False

if FLAG_ALL:
    CURRENT_TRIAL_NAME = "v2_eval_20_3"
    CURRENT_TRIAL_NAME2 = "v2_eval_20-40"
    CURRENT_TRIAL_NAME3 = "v2_eval_60-40"
    CURRENT_TRIAL_NAME4 = "v2_eval_100-35"
    CURRENT_TRIAL_NAME5 = "v2_eval_0-60"
    CURRENT_TRIAL_NAME6 = "v2_eval_60-75"
    CURRENT_TRIAL_NAME7 = "v2_eval_20-115"
    CURRENT_TRIAL_NAME8 = "v2_eval_0-135"


    
    clean_results1 = load_data(CURRENT_TRIAL_NAME, filter_flag=True, collect_results=COLLECT_RESULTS)
    clean_results2 = load_data(CURRENT_TRIAL_NAME2, collect_results=COLLECT_RESULTS)
    clean_results3 = load_data(CURRENT_TRIAL_NAME3, collect_results=COLLECT_RESULTS)
    clean_results4 = load_data(CURRENT_TRIAL_NAME4, collect_results=COLLECT_RESULTS)
    clean_results5 = load_data(CURRENT_TRIAL_NAME5, collect_results=COLLECT_RESULTS)
    clean_results6 = load_data(CURRENT_TRIAL_NAME6, collect_results=COLLECT_RESULTS)
    clean_results7 = load_data(CURRENT_TRIAL_NAME7, collect_results=COLLECT_RESULTS)
    clean_results8 = load_data(CURRENT_TRIAL_NAME8, collect_results=COLLECT_RESULTS)
   
    
    # clean_results5,clean_results6,
    clean_results = pd.concat([clean_results1,clean_results2,clean_results3, clean_results4,clean_results7,clean_results8])
    print(clean_results.shape)
else:
    CURRENT_TRIAL_NAME4 = "v2_eval_20_3"
    clean_results = load_data(CURRENT_TRIAL_NAME)

# clean_results.to_csv("../results/all_results.csv",sep=",",index=False)
clean_results.head()

(1080, 12)


,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error,env_idx
0,gpt-3.5-turbo-0125,react-1,0.0,cool,0,1,0,49,12,0,0,0
1,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,13,0,0,1
2,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,33,0,0,2
3,gpt-3.5-turbo-0125,react-1,0.0,put,0,1,0,49,26,0,0,3
4,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,39,2,0,4


In [543]:
# This is the table that shows results per env_type
out = clean_results.groupby(["model","keys_removed","env_type"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].count().reset_index()["success"]
out["success_std"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].std().reset_index()["success"]
out["done_std"] = clean_results.groupby(["model","keys_removed","env_type"])["done"].std().reset_index()["done"]
out_header_order = ["model", "keys_removed","temperature","env_type","total","success","success_std","done","done_std","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
# Adding a few more scores.
out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["success_percentage"] = out["success"] / out["total"]
out["success_percentage_done"] = out["success"] / out["done"]
out["done_percentage"] = out["done"] / out["total"]
out["normalised_total"] = out["total"] / 135
out["normalised_done"] = out["done"] / out["total"]

x = out.groupby(["env_type"])
x.head()

,model,keys_removed,temperature,env_type,total,success,success_std,done,done_std,early_stop,num_of_steps,num_nothing_happens,num_repetitions,normalised_nothing,normalised_reps,success_percentage,success_percentage_done,done_percentage,normalised_total,normalised_done
0,gpt-3.5-turbo-0125,long,0.0,clean,31,17,0.505879,25,0.401610,6,735,325,46,0.442177,0.062585,0.548387,0.680000,0.806452,0.229630,0.806452
1,gpt-3.5-turbo-0125,long,0.0,cool,21,2,0.300793,11,0.511766,10,751,296,83,0.394141,0.110519,0.095238,0.181818,0.523810,0.155556,0.523810
2,gpt-3.5-turbo-0125,long,0.0,examine,18,5,0.460889,13,0.460889,5,553,290,54,0.524412,0.097649,0.277778,0.384615,0.722222,0.133333,0.722222
3,gpt-3.5-turbo-0125,long,0.0,heat,23,13,0.506870,20,0.344350,3,521,270,35,0.518234,0.067179,0.565217,0.650000,0.869565,0.170370,0.869565
4,gpt-3.5-turbo-0125,long,0.0,put,25,5,0.408248,14,0.506623,11,760,379,84,0.498684,0.110526,0.200000,0.357143,0.560000,0.185185,0.560000
5,gpt-3.5-turbo-0125,long,0.0,puttwo,17,1,0.242536,8,0.514496,9,644,264,82,0.409938,0.127329,0.058824,0.125000,0.470588,0.125926,0.470588
6,gpt-3.5-turbo-0125,long-2,0.0,clean,31,16,0.508001,22,0.461414,9,690,221,60,0.320290,0.086957,0.516129,0.727273,0.709677,0.229630,0.709677
7,gpt-3.5-turbo-0125,long-2,0.0,cool,21,8,0.497613,15,0.462910,6,634,278,53,0.438486,0.083596,0.380952,0.533333,0.714286,0.155556,0.714286
8,gpt-3.5-turbo-0125,long-2,0.0,examine,18,1,0.235702,7,0.501631,11,504,469,133,0.930556,0.263889,0.055556,0.142857,0.388889,0.133333,0.388889
9,gpt-3.5-turbo-0125,long-2,0.0,heat,23,8,0.486985,11,0.510754,12,450,185,67,0.411111,0.148889,0.347826,0.727273,0.478261,0.170370,0.478261


In [544]:
out.to_csv("../results/pandas_output_per_env_type.csv",sep="&",index=False)

In [545]:
print(out[["model", "keys_removed","env_type","total","success"]])

                 model    keys_removed env_type  total  success
0   gpt-3.5-turbo-0125            long    clean     31       17
1   gpt-3.5-turbo-0125            long     cool     21        2
2   gpt-3.5-turbo-0125            long  examine     18        5
3   gpt-3.5-turbo-0125            long     heat     23       13
4   gpt-3.5-turbo-0125            long      put     25        5
5   gpt-3.5-turbo-0125            long   puttwo     17        1
6   gpt-3.5-turbo-0125          long-2    clean     31       16
7   gpt-3.5-turbo-0125          long-2     cool     21        8
8   gpt-3.5-turbo-0125          long-2  examine     18        1
9   gpt-3.5-turbo-0125          long-2     heat     23        8
10  gpt-3.5-turbo-0125          long-2      put     25       12
11  gpt-3.5-turbo-0125          long-2   puttwo     17        4
12  gpt-3.5-turbo-0125         react-1    clean     31        1
13  gpt-3.5-turbo-0125         react-1     cool     21        0
14  gpt-3.5-turbo-0125         react-1  

In [546]:
def create_pivot(variable="success"):# pivot1 = out.pivot(index=["keys_removed"],columns=["env_type"], values=["success"])
    pivot_table = pd.pivot_table(out, index=["model","keys_removed"],columns=["env_type"], aggfunc={variable:"first"})
    pivot_table.columns = pivot_table.columns.droplevel(0)
    # pivot1.columns.name=None
    piv_table = pivot_table.reset_index()
    return piv_table

piv1 = create_pivot("success")
piv1.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,17,2,5,13,5,1
1,gpt-3.5-turbo-0125,long-2,16,8,1,8,12,4
2,gpt-3.5-turbo-0125,react-1,1,0,7,7,1,0
3,gpt-3.5-turbo-0125,react-2,7,1,12,13,3,7
4,gpt-3.5-turbo-0125,short,16,3,7,11,7,0


In [547]:
pivot_done = create_pivot("done")
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,25,11,13,20,14,8
1,gpt-3.5-turbo-0125,long-2,22,15,7,11,14,8
2,gpt-3.5-turbo-0125,react-1,28,20,14,23,25,17
3,gpt-3.5-turbo-0125,react-2,27,16,14,20,24,14
4,gpt-3.5-turbo-0125,short,19,11,12,18,13,5


In [548]:
pivot_done = create_pivot("num_repetitions")
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,46,83,54,35,84,82
1,gpt-3.5-turbo-0125,long-2,60,53,133,67,68,62
2,gpt-3.5-turbo-0125,react-1,26,16,33,7,13,3
3,gpt-3.5-turbo-0125,react-2,43,41,27,37,16,22
4,gpt-3.5-turbo-0125,short,86,64,46,42,80,75


In [549]:
pivot_done = create_pivot("num_nothing_happens")
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,325,296,290,270,379,264
1,gpt-3.5-turbo-0125,long-2,221,278,469,185,214,184
2,gpt-3.5-turbo-0125,react-1,750,541,273,365,609,359
3,gpt-3.5-turbo-0125,react-2,479,486,135,265,520,175
4,gpt-3.5-turbo-0125,short,284,399,301,226,387,244


In [550]:
pivot_done = create_pivot("success_percentage") #higher is better
# pivot_done.head()
pivot_done

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.548387,0.095238,0.277778,0.565217,0.20,0.058824
1,gpt-3.5-turbo-0125,long-2,0.516129,0.380952,0.055556,0.347826,0.48,0.235294
2,gpt-3.5-turbo-0125,react-1,0.032258,0.000000,0.388889,0.304348,0.04,0.000000
3,gpt-3.5-turbo-0125,react-2,0.225806,0.047619,0.666667,0.565217,0.12,0.411765
4,gpt-3.5-turbo-0125,short,0.516129,0.142857,0.388889,0.478261,0.28,0.000000
5,gpt-3.5-turbo-0125,short-1-swaped,0.193548,0.285714,0.111111,0.173913,0.40,0.058824
6,gpt-3.5-turbo-0125,short-2,0.483871,0.380952,0.222222,0.478261,0.32,0.176471
7,gpt-3.5-turbo-0125,short-2-swaped,0.580645,0.571429,0.166667,0.478261,0.68,0.176471


In [551]:
pivot_done = create_pivot("normalised_nothing") #lower is better (this shows how many of the actions result in "nothing happens"
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.442177,0.394141,0.524412,0.518234,0.498684,0.409938
1,gpt-3.5-turbo-0125,long-2,0.320290,0.438486,0.930556,0.411111,0.422091,0.362919
2,gpt-3.5-turbo-0125,react-1,0.530035,0.537239,0.473958,0.433492,0.514793,0.430972
3,gpt-3.5-turbo-0125,react-2,0.402183,0.526544,0.402985,0.433007,0.471869,0.281804
4,gpt-3.5-turbo-0125,short,0.453674,0.547325,0.582205,0.432122,0.533793,0.427320


In [552]:
pivot_done = create_pivot("normalised_done") #higher is better (this shows how many reached done, without early stop)
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.806452,0.523810,0.722222,0.869565,0.56,0.470588
1,gpt-3.5-turbo-0125,long-2,0.709677,0.714286,0.388889,0.478261,0.56,0.470588
2,gpt-3.5-turbo-0125,react-1,0.903226,0.952381,0.777778,1.000000,1.00,1.000000
3,gpt-3.5-turbo-0125,react-2,0.870968,0.761905,0.777778,0.869565,0.96,0.823529
4,gpt-3.5-turbo-0125,short,0.612903,0.523810,0.666667,0.782609,0.52,0.294118


In [553]:
pivot_done = create_pivot("success_percentage_done") #higher is better (this show successful run out of all done [not total]
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.680000,0.181818,0.384615,0.650000,0.357143,0.125
1,gpt-3.5-turbo-0125,long-2,0.727273,0.533333,0.142857,0.727273,0.857143,0.500
2,gpt-3.5-turbo-0125,react-1,0.035714,0.000000,0.500000,0.304348,0.040000,0.000
3,gpt-3.5-turbo-0125,react-2,0.259259,0.062500,0.857143,0.650000,0.125000,0.500
4,gpt-3.5-turbo-0125,short,0.842105,0.272727,0.583333,0.611111,0.538462,0.000


In [554]:
pivot_done = create_pivot("normalised_reps") #lower is better (this shows number of reps out of all steps, mind you we cut after 5 exact reps)
pivot_done.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,0.062585,0.110519,0.097649,0.067179,0.110526,0.127329
1,gpt-3.5-turbo-0125,long-2,0.086957,0.083596,0.263889,0.148889,0.134122,0.122288
2,gpt-3.5-turbo-0125,react-1,0.018375,0.015889,0.057292,0.008314,0.010989,0.003601
3,gpt-3.5-turbo-0125,react-2,0.036104,0.044420,0.080597,0.060458,0.014519,0.035427
4,gpt-3.5-turbo-0125,short,0.137380,0.087791,0.088975,0.080306,0.110345,0.131349


In [555]:
pivot2 = pd.pivot_table(out, columns=["env_type"], aggfunc={"total":"first"})
pivot2["keys_removed"] = "total"
piv2 = pivot2.reset_index()
piv2.head()

env_type,index,clean,cool,examine,heat,put,puttwo,keys_removed
0,total,31,21,18,23,25,17,total


In [556]:
pivot3 = pd.pivot_table(out, columns=["env_type"], aggfunc={"normalised_total":"first"})
pivot3["keys_removed"] = "total"
piv3 = pivot3.reset_index()
piv3.head()

env_type,index,clean,cool,examine,heat,put,puttwo,keys_removed
0,normalised_total,0.22963,0.155556,0.133333,0.17037,0.185185,0.125926,total


In [557]:
new = pd.concat([piv1,piv2])
new = new.drop("index",axis=1)
new.head()

env_type,model,keys_removed,clean,cool,examine,heat,put,puttwo
0,gpt-3.5-turbo-0125,long,17,2,5,13,5,1
1,gpt-3.5-turbo-0125,long-2,16,8,1,8,12,4
2,gpt-3.5-turbo-0125,react-1,1,0,7,7,1,0
3,gpt-3.5-turbo-0125,react-2,7,1,12,13,3,7
4,gpt-3.5-turbo-0125,short,16,3,7,11,7,0


In [558]:
# This is the table that shows total results.
out = clean_results.groupby(["model","keys_removed"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","temperature","total","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["success_percentage"] = out["success"] / out["total"]
out["success_percentage_done"] = out["success"] / out["done"]
out["done_percentage"] = out["done"] / out["total"]
x = out.groupby("keys_removed")
x.head()

,model,keys_removed,temperature,total,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,normalised_nothing,normalised_reps,success_percentage,success_percentage_done,done_percentage
0,gpt-3.5-turbo-0125,long,0.0,135,43,91,44,3964,1824,384,0.460141,0.096872,0.318519,0.472527,0.674074
1,gpt-3.5-turbo-0125,long-2,0.0,135,49,77,58,3292,1551,443,0.471142,0.134569,0.362963,0.636364,0.570370
2,gpt-3.5-turbo-0125,react-1,0.0,135,16,127,8,5856,2897,98,0.494706,0.016735,0.118519,0.125984,0.940741
3,gpt-3.5-turbo-0125,react-2,0.0,135,43,115,20,4784,2060,186,0.430602,0.038880,0.318519,0.373913,0.851852
4,gpt-3.5-turbo-0125,short,0.0,135,44,78,57,3691,1841,393,0.498781,0.106475,0.325926,0.564103,0.577778
5,gpt-3.5-turbo-0125,short-1-swaped,0.0,135,29,80,55,4407,2351,496,0.533469,0.112548,0.214815,0.362500,0.592593
6,gpt-3.5-turbo-0125,short-2,0.0,135,49,88,47,3425,1455,350,0.424818,0.102190,0.362963,0.556818,0.651852
7,gpt-3.5-turbo-0125,short-2-swaped,0.0,135,64,98,37,3274,1582,354,0.483201,0.108125,0.474074,0.653061,0.725926


In [559]:
summary_header = ["keys_removed","success_percentage","success_percentage_done","done_percentage","normalised_reps","normalised_nothing"]
outsummary = out[summary_header]
x = outsummary.groupby("keys_removed")
x.head()

,keys_removed,success_percentage,success_percentage_done,done_percentage,normalised_reps,normalised_nothing
0,long,0.318519,0.472527,0.674074,0.096872,0.460141
1,long-2,0.362963,0.636364,0.570370,0.134569,0.471142
2,react-1,0.118519,0.125984,0.940741,0.016735,0.494706
3,react-2,0.318519,0.373913,0.851852,0.038880,0.430602
4,short,0.325926,0.564103,0.577778,0.106475,0.498781
5,short-1-swaped,0.214815,0.362500,0.592593,0.112548,0.533469
6,short-2,0.362963,0.556818,0.651852,0.102190,0.424818
7,short-2-swaped,0.474074,0.653061,0.725926,0.108125,0.483201


In [465]:
out.to_csv("../results/pandas_output_all_results.csv",sep="&",index=False)

In [466]:
# This is the table that shows results per env_id
out = clean_results.groupby(["model","env_idx","keys_removed","env_type"]).sum().reset_index()
out_header_order = ["model", "keys_removed","temperature","env_idx","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]
x = out.groupby("env_idx")
x.head()

,model,keys_removed,temperature,env_idx,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,long,0.0,0,cool,0,0,1,33,21,7
1,gpt-3.5-turbo-0125,long-1-swaped,0.0,0,cool,0,0,1,47,24,9
2,gpt-3.5-turbo-0125,long-2,0.0,0,cool,0,1,0,49,43,0
3,gpt-3.5-turbo-0125,long-2-swaped,0.0,0,cool,1,1,0,16,4,2
4,gpt-3.5-turbo-0125,react-1,0.0,0,cool,0,1,0,49,12,0
...,...,...,...,...,...,...,...,...,...,...,...
1608,gpt-3.5-turbo-0125,long,0.0,134,put,0,0,1,43,12,5
1609,gpt-3.5-turbo-0125,long-1-swaped,0.0,134,put,1,1,0,4,0,0
1610,gpt-3.5-turbo-0125,long-2,0.0,134,put,1,1,0,7,1,0
1611,gpt-3.5-turbo-0125,long-2-swaped,0.0,134,put,1,1,0,4,0,0


In [467]:
pivot_per_env = out.pivot(index=["env_idx","env_type"],columns=["keys_removed"], values=["success"])
pivot_per_env

success                                             \
keys_removed        long long-1-swaped long-2 long-2-swaped react-1   
env_idx env_type                                                      
0       cool           0             0      0             1       0   
1       clean          0             0      1             0       0   
2       clean          1             0      1             1       0   
3       put            0             0      0             1       0   
4       clean          0             0      0             0       0   
...                  ...           ...    ...           ...     ...   
130     examine        1             0      0             1       0   
131     heat           0             1      0             1       1   
132     clean          0             0      0             0       0   
133     examine        0             0      0             0       0   
134     put            0             1      1             1       0   

                                                                             \
keys_removed     react-1-swaped react-2 react-2-swaped short short-1-swaped   
env_idx env_type                                                              
0       cool                  0       1              0     0              0   
1       clean                 0       0              0     1              0   
2       clean                 0       0              0     0              0   
3       put                   0       0              0     1              0   
4       clean                 0       0              0     0              1   
...                         ...     ...            ...   ...            ...   
130     examine               0       0              1     1              0   
131     heat                  0       1              1     1              0   
132     clean                 0       0              0     0              0   
133     examine               0       1              0     0              0   
134     put                   0       0              0     0              1   

                                         
keys_removed     short-2 short-2-swaped  
env_idx env_type                         
0       cool           1              1  
1       clean          1              0  
2       clean          1              0  
3       put            0              1  
4       clean          0              0  
...                  ...            ...  
130     examine        0              1  
131     heat           1              1  
132     clean          0              1  
133     examine        0              0  
134     put            1              1  

[135 rows x 12 columns]

In [468]:
p2 = pivot_per_env.droplevel(0,axis=1)
p3 = p2.reset_index()

In [479]:
def create_analysis(key1, key2, p3):
    k1vsk2 = key1+" wins"
    k2vsk1 = key2+" wins"
    k1isk2 = "Draw"  
    
    x = pd.DataFrame()
    x["env_type"] = p3["env_type"].values

    x[k1vsk2] = (p3[key1] > p3[key2]) * 1.0
    x[k2vsk1] = (p3[key1] < p3[key2]) * 1.0
    x[k1isk2] = (p3[key1] == p3[key2]) * 1.0
    pivot = pd.pivot_table(x, columns=["env_type"], aggfunc={k1vsk2:"sum", k2vsk1:"sum", k1isk2:"sum"})
    return pivot

In [480]:
create_analysis("long","long-1-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,18.0,13.0,11.0,12.0,16.0,14.0
long wins,13.0,0.0,5.0,10.0,3.0,1.0
long-1-swaped wins,0.0,8.0,2.0,1.0,6.0,2.0


In [501]:
create_analysis("long","long-2",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,20.0,11.0,12.0,16.0,16.0,14.0
long wins,6.0,2.0,5.0,6.0,1.0,0.0
long-2 wins,5.0,8.0,1.0,1.0,8.0,3.0


In [503]:
create_analysis("long","long-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,22.0,13.0,13.0,17.0,12.0,16.0
long wins,7.0,0.0,2.0,5.0,1.0,0.0
long-2-swaped wins,2.0,8.0,3.0,1.0,12.0,1.0


In [502]:
create_analysis("long-1-swaped","long-2",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,19.0,13.0,15.0,15.0,17.0,13.0
long-1-swaped wins,0.0,5.0,2.0,2.0,2.0,1.0
long-2 wins,12.0,3.0,1.0,6.0,6.0,3.0


In [504]:
create_analysis("long-1-swaped","long-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,21.0,15.0,12.0,16.0,15.0,13.0
long-1-swaped wins,1.0,3.0,1.0,1.0,1.0,2.0
long-2-swaped wins,9.0,3.0,5.0,6.0,9.0,2.0


In [505]:
create_analysis("long-2","long-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,25.0,11.0,11.0,18.0,17.0,13.0
long-2 wins,5.0,4.0,1.0,2.0,2.0,3.0
long-2-swaped wins,1.0,6.0,6.0,3.0,6.0,1.0


In [494]:
create_analysis("short","short-1-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,17.0,18.0,11.0,14.0,17.0,15.0
short wins,12.0,0.0,6.0,7.0,2.0,0.0
short-1-swaped wins,2.0,3.0,1.0,2.0,6.0,2.0


In [495]:
create_analysis("short-2","short-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,20.0,15.0,11.0,19.0,14.0,15.0
short-2 wins,4.0,1.0,4.0,2.0,1.0,1.0
short-2-swaped wins,7.0,5.0,3.0,2.0,10.0,1.0


In [496]:
create_analysis("react-1","react-1-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,30.0,19.0,9.0,14.0,24.0,11.0
react-1 wins,1.0,0.0,7.0,7.0,0.0,0.0
react-1-swaped wins,0.0,2.0,2.0,2.0,1.0,6.0


In [497]:
create_analysis("react-2","react-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,26.0,13.0,12.0,14.0,20.0,12.0
react-2 wins,2.0,1.0,2.0,4.0,2.0,2.0
react-2-swaped wins,3.0,7.0,4.0,5.0,3.0,3.0


In [498]:
create_analysis("long-2-swaped","react-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,19.0,12.0,8.0,12.0,13.0,9.0
long-2-swaped wins,8.0,6.0,1.0,3.0,12.0,1.0
react-2-swaped wins,4.0,3.0,9.0,8.0,0.0,7.0


In [499]:
create_analysis("react-2-swaped","short-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,19.0,12.0,5.0,16.0,12.0,8.0
react-2-swaped wins,1.0,2.0,12.0,5.0,0.0,7.0
short-2-swaped wins,11.0,7.0,1.0,2.0,13.0,2.0


In [500]:
create_analysis("long-2-swaped","short-2-swaped",p3)

env_type,clean,cool,examine,heat,put,puttwo
Draw,21.0,19.0,13.0,19.0,18.0,16.0
long-2-swaped wins,2.0,0.0,4.0,1.0,3.0,0.0
short-2-swaped wins,8.0,2.0,1.0,3.0,4.0,1.0


In [585]:
deepdive = load_data("deepdive_results.csv",collect_results=True, override_filepath=True)
deepdive.head()

,total_nothing,total_not_valid_nothing,put_not_valid_nothing,hallucinated_not_valid_nothing,hallucinated_example,fake_not_h_not_valid_nothing,fake_command_example,total_valid_nothing,take_valid_nothing,valid_nothing_example,...,num_json_dsnt_load,num_multi_json,num_no_json,num_json_and_text,error,early_stop,keys_removed,additional_prompt_annotation,trace_file,prompt_file
0,12,6,6,0,NO_EXAMPLE,6,put tomato 1 on stoveburner 1,6,6,NO EXAMPLE,...,50,0,50,0,1,1,react-1,NaN,game_logs/alfworld_eval_v2_eval_20_3/OpenAITex...,game_logs/alfworld_eval_v2_eval_20_3/prompt_18...
1,13,13,13,0,NO_EXAMPLE,13,put spatula 1 in microwave 1,0,0,NO EXAMPLE,...,50,0,50,0,1,1,react-1,NaN,game_logs/alfworld_eval_v2_eval_20_3/OpenAITex...,game_logs/alfworld_eval_v2_eval_20_3/prompt_18...
2,32,10,10,0,NO_EXAMPLE,10,put plate 1 back in cabinet 1,22,18,go to diningtable 1,...,50,0,50,0,1,1,react-1,NaN,game_logs/alfworld_eval_v2_eval_20_3/OpenAITex...,game_logs/alfworld_eval_v2_eval_20_3/prompt_18...
3,25,16,1,0,NO_EXAMPLE,16,pick up cellphone 3 from shelf 11,9,6,go to shelf 3,...,50,0,50,0,1,1,react-1,NaN,game_logs/alfworld_eval_v2_eval_20_3/OpenAITex...,game_logs/alfworld_eval_v2_eval_20_3/prompt_18...
4,38,35,10,20,put cloth 1 on top of cloth 2 in cabinet 4,15,put cloth 1 with cloth 2 in cabinet 4,3,3,NO EXAMPLE,...,50,0,50,0,1,1,react-1,NaN,game_logs/alfworld_eval_v2_eval_20_3/OpenAITex...,game_logs/alfworld_eval_v2_eval_20_3/prompt_18...


In [586]:
# This is the table that shows total results.
out = deepdive.groupby(["model","keys_removed"]).sum().reset_index()
out["total"] = deepdive.groupby(["model","keys_removed"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","total","success","done","num_of_steps","total_nothing","total_not_valid_nothing","hallucinated_not_valid_nothing","fake_not_h_not_valid_nothing","put_not_valid_nothing","total_valid_nothing","take_valid_nothing"]
out = out[out_header_order]
# out["normalised_nothing"] = out["num_nothing_happens"] / out["num_of_steps"]
# out["normalised_reps"] = out["num_repetitions"] / out["num_of_steps"]
out["normalised_nothing"] = out["total_nothing"] / out["num_of_steps"]
out["normalised_not_valid_nothing"] = out["total_not_valid_nothing"] / out["num_of_steps"]
out["proportion_of_not_valid_in_nothing"] = out["total_not_valid_nothing"] / out["total_nothing"]
out["proportion_of_valid_in_nothing"] = out["total_valid_nothing"] / out["total_nothing"]
out["proportion_of_hallucinated_in_not_valid"] = out["hallucinated_not_valid_nothing"] / out["total_not_valid_nothing"]
out["proportion_of_fake_in_not_valid"] = out["fake_not_h_not_valid_nothing"] / out["total_not_valid_nothing"]

# out["success_percentage"] = out["success"] / out["total"]
# out["success_percentage_done"] = out["success"] / out["done"]
# out["done_percentage"] = out["done"] / out["total"]
x = out.groupby("keys_removed")
x.head()

,model,keys_removed,total,success,done,num_of_steps,total_nothing,total_not_valid_nothing,hallucinated_not_valid_nothing,fake_not_h_not_valid_nothing,put_not_valid_nothing,total_valid_nothing,take_valid_nothing,normalised_nothing,normalised_not_valid_nothing,proportion_of_not_valid_in_nothing,proportion_of_valid_in_nothing,proportion_of_hallucinated_in_not_valid,proportion_of_fake_in_not_valid
0,gpt-3.5-turbo-0125,long,135,43,91,3964,1747,1155,876,279,113,592,361,0.440716,0.291372,0.661133,0.338867,0.758442,0.241558
1,gpt-3.5-turbo-0125,long-1-swaped,135,30,69,3848,1856,1223,893,330,58,633,369,0.482328,0.317827,0.658944,0.341056,0.730172,0.269828
2,gpt-3.5-turbo-0125,long-2,135,49,77,3292,1472,1009,794,215,34,463,299,0.447145,0.306501,0.685462,0.314538,0.786918,0.213082
3,gpt-3.5-turbo-0125,long-2-swaped,135,55,86,3271,1343,725,603,122,16,618,397,0.410578,0.221645,0.539836,0.460164,0.831724,0.168276
4,gpt-3.5-turbo-0125,react-1,135,16,127,5856,2815,2105,565,1540,1091,710,331,0.480704,0.359460,0.747780,0.252220,0.268409,0.731591
5,gpt-3.5-turbo-0125,react-1-swaped,135,14,117,5810,2617,2002,604,1398,879,615,247,0.450430,0.344578,0.764998,0.235002,0.301698,0.698302
6,gpt-3.5-turbo-0125,react-2,135,43,115,4784,1996,1499,385,1114,836,497,228,0.417224,0.313336,0.751002,0.248998,0.256838,0.743162
7,gpt-3.5-turbo-0125,react-2-swaped,135,55,120,4380,1756,1328,354,974,783,428,211,0.400913,0.303196,0.756264,0.243736,0.266566,0.733434
8,gpt-3.5-turbo-0125,short,135,44,78,3691,1756,1152,789,363,144,604,388,0.475752,0.312111,0.656036,0.343964,0.684896,0.315104
9,gpt-3.5-turbo-0125,short-1-swaped,135,33,63,3852,2051,1337,860,477,72,714,443,0.532451,0.347092,0.651877,0.348123,0.643231,0.356769
